In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import pickle
import seaborn as sns
from sklearn.model_selection import train_test_split
from collections import defaultdict
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import tqdm 
import os
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, KFold, GridSearchCV
from surprise import NormalPredictor, BaselineOnly, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD, NMF, SlopeOne, CoClustering    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

with open('../data/data_for_model/data_for_model.pkl', 'rb') as f:
    # Load the contents of the file using pickle.load()
    data_for_model = pickle.load(f)

In [3]:
algo = KNNBasic()

In [4]:
names = ['recalls_list','precs_list','result_all']
names_dict = defaultdict(list)

In [6]:
#loop to load all pickle files to later turn into dataframe
data_dir = '../data/'
folder_list = [i for i in os.listdir(data_dir) if i not in ['.ipynb_checkpoints','input','data_for_model','no_null_user_ratings']]
for i in folder_list:
    x = f"{data_dir}{i}"
    for enum_num,enum_name in enumerate(names):
        file_path = f"{x}/{enum_num}.pickle"
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
            names_dict[enum_name].append(data)


In [7]:
averages = {}
for sublist in names_dict['result_all']:
    for algorithm, score in sublist:
        if algorithm not in averages:
            averages[algorithm] = score
        else:
            averages[algorithm].append(score)


In [8]:
averages

{'SVD': [1.0213712081874704,
  [1.0204828673113941],
  [1.0203655120368236],
  [1.0200507047167389],
  [1.0200749272648115]],
 'NonNegative Matrix Factorization': [1.7844759821295415,
  [1.777024081528765],
  [1.7885889749663555],
  [1.7871589997098034],
  [1.785430450313568]],
 'Slope One': [1.0174268302254594,
  [1.0175506091427093],
  [1.0174059117435126],
  [1.0175889050438958],
  [1.0175749963301826]],
 'Co-clustering': [1.045843777282942,
  [1.0469009270176166],
  [1.045874629570217],
  [1.0448459966189174],
  [1.0448217456155375]],
 'Baseline': [1.0205603642904433,
  [1.0204669150813224],
  [1.0205703614676553],
  [1.0206715940241646],
  [1.0205877974894502]],
 'Normal_predictor': [1.851355658968064,
  [1.854540711200352],
  [1.8519912733150865],
  [1.8551386751747798],
  [1.8539058727766673]],
 'KNNBasic': [1.0719991243168838,
  [1.0720212512429264],
  [1.0720624660482645],
  [1.0717710467632777],
  [1.0717790416347135]],
 'KNNBaseline': [1.0162666704578496,
  [1.01631401694101

In [9]:
rmse_dict = {}
for algorithm, scores in averages.items():
    average_score = sum(scores) / len(scores)
    rmse_dict[algorithm] =  average_score[0]

In [10]:
rmse_dict

{'SVD': 1.0204690439034478,
 'NonNegative Matrix Factorization': 1.7845356977296067,
 'Slope One': 1.017509450497152,
 'Co-clustering': 1.045657415221046,
 'Baseline': 1.020571406470607,
 'Normal_predictor': 1.85338643828699,
 'KNNBasic': 1.0719265860012133,
 'KNNBaseline': 1.0161946293396178,
 'KNNWithMeans': 1.0323789311256122,
 'KNNWithZScore': 1.0320189830093287}

In [11]:
recalls_dict = {model_name: recall_score for recall_list in 
                names_dict['recalls_list'] 
                for model_name, recall_score in recall_list}


In [12]:
precs_dict = {model_name: recall_score for recall_list in 
                names_dict['precs_list'] 
                for model_name, recall_score in recall_list}

In [13]:
rmse_dict

{'SVD': 1.0204690439034478,
 'NonNegative Matrix Factorization': 1.7845356977296067,
 'Slope One': 1.017509450497152,
 'Co-clustering': 1.045657415221046,
 'Baseline': 1.020571406470607,
 'Normal_predictor': 1.85338643828699,
 'KNNBasic': 1.0719265860012133,
 'KNNBaseline': 1.0161946293396178,
 'KNNWithMeans': 1.0323789311256122,
 'KNNWithZScore': 1.0320189830093287}

In [14]:
dict_list = [recalls_dict,precs_dict,rmse_dict]
dataframe_results = pd.DataFrame(dict_list).T

In [15]:
dataframe_results.rename(columns={0:'recall_at_k',1:'precision_at_k',2:'average_rmse'},inplace=True)

In [17]:
dataframe_final = dataframe_results[['precision_at_k','recall_at_k','average_rmse']].sort_values('precision_at_k',ascending=False)

In [77]:
dataframe_final

,precision_at_k,recall_at_k,average_rmse
KNNBasic,0.698465,0.428652,1.071927
SVD,0.698177,0.390089,1.020469
KNNBaseline,0.688521,0.356677,1.016195
Baseline,0.686130,0.367994,1.020571
KNNWithZScore,0.683345,0.367066,1.032019
Slope One,0.679875,0.360517,1.017509
Co-clustering,0.665922,0.333746,1.045657
KNNWithMeans,0.661810,0.334689,1.032379
Normal_predictor,0.465807,0.314209,1.853386
NonNegative Matrix Factorization,0.087494,0.009557,1.784536


In [ ]:
# dataframe_final.to_markdown()

As we are more concerned from a business standpoint of the overall precision of our model (the number of relevant predictions to our user) we use that as our most important metric as the difference between models is neglible at 
a 0.5% change in rmse.

Our strongest two models were SVD and KNNBasic.

When choosing between two recommendation models, SVD and KNNBasic we can see  and their overall RMSE and Precision@K are similar, but KNNBasic has a better Recall@K we chose KNNBasic over SVD by considering the following:

Recall is a measure of the fraction of relevant items that were recommended to the user, out of all relevant items. A higher recall means that more relevant items were recommended to the user.

Overall, if recall is more important for your application and you have a high degree of user-item sparsity, KNNBasic is the better as compared to SVD.

## Final model

As concluded above, we go with KNNBasic based on the strongest precision and recall score.

In [20]:
model = KNNBasic()
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(data_for_model[['Username','BGGId', 'Rating']],reader)

In [21]:
train=data.build_full_trainset()
model.fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [28]:
games_csv2 = pd.read_csv('../data/no_null_user_ratings/games_after_2010.csv')

In [23]:
import random
user_list = []
game_list = data_for_model[data_for_model['Username'] == 'Evabelle']['BGGId'].to_list()

In [29]:
not_user_list = [i for i in games_csv2['BGGId'].to_list() if i not in game_list]

In [30]:
predict_list = [model.predict(uid='Evabelle', iid=i) for i in not_user_list]
est = [i.est for i in predict_list if i.details['was_impossible'] !=True]
iids = [i.iid for i in predict_list if i.details['was_impossible'] !=True]

test_df = pd.DataFrame({'est': est,
              'BGGId': iids}
            )

In [32]:
df_test = pd.merge(test_df, games_csv2, on='BGGId')

In [80]:
df_test[['est','BGGId','Name']].sort_values('est',ascending=False).reset_index(drop=True).head(10)

,est,BGGId,Name
0,9.967750,284121,Uprising: Curse of the Last Emperor
1,9.561072,342942,Ark Nova
2,9.546281,295785,Euthia: Torment of Resurrection
3,9.252500,341169,Great Western Trail (Second Edition)
4,9.146072,237828,Anno Domini 1666
5,9.104490,249277,Brazil: Imperial
6,9.095526,209951,Thunder in the East
7,8.948703,299317,Aeon's End: Outcasts
8,8.934255,277659,Final Girl
9,8.932701,299659,Clash of Cultures: Monumental Edition


## Demoing on new user

In [45]:
import random
bgg_id_demo = []
rating_demo = []
for i in range(5):
    bgg_id_demo.append(random.choice(data_for_model['BGGId']))
    rating_demo.append(random.uniform(5.0, 10.0))
        
        

In [46]:
rating_demo

[7.7757680277325285,
 7.090790479541342,
 5.556141574921927,
 7.476484376301415,
 7.600644140707615]

From a deployment standpoint, we need to make sure that our model is able to output recommendations.

In [49]:
df_new_user = pd.DataFrame([], columns=['BGGId','Rating','Username'])
    

In [52]:
df_new_user['BGGId'] = bgg_id_demo
df_new_user['Rating'] = rating_demo
df_new_user['Username'] = 'demo_user'

In [56]:
demo_df = pd.concat([data_for_model,df_new_user])

In [58]:
model = KNNBasic()
reader = Reader(rating_scale=(1, 10))
data_2 = Dataset.load_from_df(demo_df[['Username','BGGId', 'Rating']],reader)

In [60]:
train=data_2.build_full_trainset()
model.fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [61]:
predict_list = [model.predict(uid='demo_user', iid=i) for i in not_user_list]
est = [i.est for i in predict_list if i.details['was_impossible'] !=True]
iids = [i.iid for i in predict_list if i.details['was_impossible'] !=True]

demo_test_df = pd.DataFrame({'est': est,
              'BGGId': iids}
            )

In [63]:
df_test_2 = pd.merge(demo_test_df, games_csv2, on='BGGId')

In [76]:
df_test_2[['est','BGGId','Name','']].sort_values('est',ascending=False).reset_index(drop=True).head(10)

,est,BGGId,Name
0,9.967750,284121,Uprising: Curse of the Last Emperor
1,9.561072,342942,Ark Nova
2,9.546281,295785,Euthia: Torment of Resurrection
3,9.252500,341169,Great Western Trail (Second Edition)
4,9.146072,237828,Anno Domini 1666
5,9.104490,249277,Brazil: Imperial
6,9.095526,209951,Thunder in the East
7,8.948703,299317,Aeon's End: Outcasts
8,8.934255,277659,Final Girl
9,8.932701,299659,Clash of Cultures: Monumental Edition


## Conclusion

Our model is a powerful tool that is capable of providing highly accurate recommendations for users, making it an invaluable asset to the platform. As users become more active and leave more reviews, our model's ability to provide insightful and relevant recommendations will only increase, resulting in even greater user satisfaction.

Furthermore, by leveraging our model to increase user engagement, the platform stands to benefit tremendously. With its ability to provide solid recommendations based on users' prior reviews, our model can help to create a more personalized and engaging experience for users, ultimately leading to greater retention and loyalty.

### Limitations

Our model did not fully utilise all data avalabile dataset in the dataset to train the recommender system, and thus acts on the assumption the interest of our user is only limited to games released in the last 5 years. 

Due to computational limitations precision was priortised over recall for this model.

Filtering the dataset to games that are reviewed more than 100 times and users with 100 or more reviews can help improve data quality and reduce noise. However, it can also limit coverage, user behavior, and lead to data sparsity, which can negatively impact the performance of the Recall@k metric, as well as a cold start problem as similarly to most collaborative filtering recommender algorithims. 

For new users, the recommender system has no historical data on which to base recommendations, and therefore cannot make accurate predictions. Similarly, for new items, the recommender system has no prior information on which to base recommendations, and therefore cannot identify which users might be interested in the item.

The model, would require constant retraining to keep up to date with all new users inorder to mantain the quality of the recommender systems.



Future work could involve a model that can account for more than the users and reviews, such as pytorch's recommendation module, as well as using big data tools such as PySpark that allows us to train models on large datasets.  